In [ ]:
import vectice as vct

vec_project = vct.connect(config="token_i.json")

### Read datasets:
 - PTY_ID_MAIN - From our BigQuery Dev environment
 - HIST_TRANS - From our BigQuery Dev environment
 - LuxAir_Accts - S3
 - OFAC_SDN - S3

In [ ]:
# Connect to BigQuery Dev using Service Account
from google.cloud import bigquery
from google.oauth2 import service_account
creds = service_account.Credentials.from_service_account_file("bq_dev_sa.json", scopes=["https://www.googleapis.com/auth/cloud-platform"])

bigquery_client = bigquery.Client(
    credentials= creds,
    project=creds.project_id
)

Query PTY_ID_MAIN and HIST_TRX from our Dev BigQUery env. Retrieving full tables, we will need to remove non US customers from the resultsets as per compliance.

In [ ]:
# Query PTY_ID_MAIN table
qry_PTY_ID_MAIN = "SELECT * FROM `solutions-engineering-363108.CUST_PTY_INFO.PTY_ID_MAIN`"
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(qry_PTY_ID_MAIN)
df_PTY_ID_MAIN = Query_Results.to_dataframe()
#View top few rows of result
df_PTY_ID_MAIN.head()

In [ ]:
# Query HIST_TRANS table
qry_HIST_TRX = "SELECT * FROM `solutions-engineering-363108.HIST_CUST_INFO.HIST_TRANS`"
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(qry_HIST_TRX)
df_HIST_TRX = Query_Results.to_dataframe()
#View top few rows of result
df_HIST_TRX.head()

Reading the two external files from our S3 bucket.

In [ ]:
# Read the external files from S3
# Create connection
from boto3 import client
from botocore import UNSIGNED
from botocore.client import Config
import s3fs

s3_client = client('s3', config=Config(signature_version=UNSIGNED), region_name='us-west-1')


In [ ]:
import pandas as pd

# Read the external files in dataframes
s3 = s3fs.S3FileSystem(anon=True)

with s3.open("vectice-examples/Samples Data/LuxAir_Accts.csv", mode="rb") as f:
    df_LuxAir_Accts = pd.read_csv(f)

with s3.open("vectice-examples/Samples Data/OFAC_SDN.csv", mode="rb") as f:
    df_OFAC_SDN = pd.read_csv(f)

Drop features that are not needed, create target, split LuxAIr_Accts.CUST_NAME into CUST_F_NAME and CUST_L_NAME.

In [ ]:
# Prep LuxAir dataset
# Remove NAs
df_LuxAir_Accts = df_LuxAir_Accts.dropna()
# Split columns
df_LuxAir_Accts[['CUST_F_NAME', 'CUST_L_NAME']] = df_LuxAir_Accts['CUST_NAME'].str.split(pat=' ', n=1,expand=True)
# Drop features
df_LuxAir_Accts = df_LuxAir_Accts.drop(columns=['CUST_NAME','CUST_PREFS'])
df_LuxAir_Accts.shape[0]



In [ ]:
# Prep PTY_ID_MAIN dataset
# Remove NAs
df_PTY_ID_MAIN = df_PTY_ID_MAIN.dropna()
# Only keep US accts
df_PTY_ID_MAIN = df_PTY_ID_MAIN.loc[df_PTY_ID_MAIN["Customer_Address_Cntry"] == 'US' ]
df_PTY_ID_MAIN.shape[0]

In [ ]:
# Join the datasets to create a clean dataset ready for modeling
df_JOINED_DS = pd.merge(df_PTY_ID_MAIN, df_LuxAir_Accts, how='inner', left_on='Customer_POB', right_on = 'CUST_CITY')

df_JOINED_DS.head()

In [ ]:
import numpy as np
df_JOINED_DS["accepted_offer"] = np.where((df_JOINED_DS['YTD_SEGMENTS'] > 54), 0, 1)
df_JOINED_DS.head()

Save the clean dataset in BQ dev

In [ ]:
table_id = 'FFBank_CoBranded_project.modelingdata'

# Since string columns use the "object" dtype, pass in a (partial) schema
# to ensure the correct BigQuery data type.
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("Customer_PTY_Id", "STRING"),
])

job = bigquery_client.load_table_from_dataframe(
    df_JOINED_DS, table_id, job_config=job_config
)

# Vectice Documentation Pipeline

In [ ]:
from vectice import Dataset, S3Resource
from vectice.models.resource import BigQueryResource

iteration = vec_project.phase("Data Analysis").iteration()

In [ ]:
# Log insights in Vectice
msg = "We dropped rows with NA values from both the LuxAir_Accts and the PTY_ID_MAIN dataframes.\n"\
"We split the LuxAir CUST_NAME into CUST_F_NAME and CUST_L_NAME, dropped CUST_NAME and CUST_PREFS.\n"

iteration.step_construct_data = msg

# Add the clean dataset to Vectice
vct_JOINED_DS = BigQueryResource (bq_client=bigquery_client, path="solutions-engineering-363108." + table_id)
iteration.step_integrate_data = Dataset.clean(name="Clean dataset", resource=vct_JOINED_DS, dataframe = df_JOINED_DS, properties={"BQ Table Name":table_id})

iteration.step_integrate_data = "We removed non US customers fromour datasets as per compliance.\n We merged the dataframes and saved the dataset on our BigQuery environemnt as \"modelingdata\"."

iteration.step_format_data = "We generated a dataset ready for modeling.\nIn order to productionalize this we will need to establish a data pipeline and automated processes."

iteration.step_certify_compliance = f"\'Customer_Address_Cntry\' unique value(s):{df_JOINED_DS.Customer_Address_Cntry.unique()}"\
    f"\nLuxAir KYC check done against the OFAC_SDN dataset, removed 1 account(s)"

iteration.complete()